In [0]:
!pip install transformers
!pip install sentence_transformers
!pip install spacy
!python -m spacy download en_core_web_lg


     |████████████████████████████████| 645kB 3.5MB/s 
     |████████████████████████████████| 890kB 16.8MB/s 
     |████████████████████████████████| 1.1MB 23.4MB/s 
     |████████████████████████████████| 3.8MB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7822b4a73e14b6f6d976f673f1c7381c86891f64676b49eff51367f3aa68e2f8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 61kB 2.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=e34246563b899c88d4317bbd86f843dde764a0417664192ca3a374024c83a2d6
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
Successfully built sentence-transformers
     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: f

In [0]:
!curl -L -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9684k  100 9684k    0     0  5184k      0  0:00:01  0:00:01 --:--:-- 5184k


In [0]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [0]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [0]:
import spacy, random
nlp = spacy.load('en_core_web_lg')

In [0]:
# Load Movielines & Conversations
movie_lines = {}
for line in open("./cornell movie-dialogs corpus/movie_lines.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        movie_lines[parts[0]] = parts[4]
    else:
        movie_lines[parts[0]] = ""

import json
responses = {}
for line in open("./cornell movie-dialogs corpus/movie_conversations.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    line_ids = json.loads(parts[3].replace("'", '"'))
    for first, second in zip(line_ids[:-1], line_ids[1:]):
        responses[first] = second

In [0]:
import numpy as np
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape


def sentence_vector_trnf(embedder, s):
    if s == "":
        s = " "
    sentence_embedding = embedder.encode([s])
    return sentence_embedding[0]

In [0]:
!pip install simpleneighbors


In [0]:
from simpleneighbors import SimpleNeighbors

In [0]:
response_sample = random.sample(list(responses.keys()), 100000)


In [0]:
#  Using GLOVE
nns = SimpleNeighbors(300)
for i, line_id in enumerate(response_sample):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()


0 L348716 Yeah, it's getting late. I could sure use a bath.
1000 L194535 Joely...
2000 L243990 You're on this...
3000 L172669 No. No. That's not it. We don't kill him. We don't kill anybody. But I like the way you're thinking. You're focused and you're trying to follow. No, what we need to do is start a fake Special Needs class.
4000 L411369 No Roger from op/ed?
5000 L68456 Yeah, it's the best one.
6000 L542789 It's closed... closed. We need one that's open.
7000 L172141 But you're my father -
8000 L391845 Aaron?
9000 L97404 Dorothy. No!
10000 L374046 'Art thou the King of the Jews'?
11000 L52540 Okay.  Then what?
12000 L185335 You damn well thought wrong.
13000 L23394 Anything else?
14000 L222118 Would-be. I'm plagued with my share of difficulties just at the moment.
15000 L490915 Is this in the line of duty, sergeant?
16000 L46758 How do you think I felt -- riding home on that seven-fourteen train?
17000 L535686 How do you know this...?
18000 L525830 What's that?
19000 L634440 Is tha

In [0]:
# Using Sentence Embedder
nns_trf = SimpleNeighbors(768)
for i, line_id in enumerate(response_sample):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_vector_trnf(embedder, line_text)
    if np.any(summary_vector):
        nns_trf.add_one(line_id, summary_vector)
nns_trf.build()

0 L348716 Yeah, it's getting late. I could sure use a bath.
1000 L194535 Joely...
2000 L243990 You're on this...
3000 L172669 No. No. That's not it. We don't kill him. We don't kill anybody. But I like the way you're thinking. You're focused and you're trying to follow. No, what we need to do is start a fake Special Needs class.
4000 L411369 No Roger from op/ed?
5000 L68456 Yeah, it's the best one.
6000 L542789 It's closed... closed. We need one that's open.
7000 L172141 But you're my father -
8000 L391845 Aaron?
9000 L97404 Dorothy. No!
10000 L374046 'Art thou the King of the Jews'?
11000 L52540 Okay.  Then what?
12000 L185335 You damn well thought wrong.
13000 L23394 Anything else?
14000 L222118 Would-be. I'm plagued with my share of difficulties just at the moment.
15000 L490915 Is this in the line of duty, sergeant?
16000 L46758 How do you think I felt -- riding home on that seven-fourteen train?
17000 L535686 How do you know this...?
18000 L525830 What's that?
19000 L634440 Is tha

In [0]:
# Glove Predictions
sentence = " Good evening, Captain."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])


Your line:
	  Good evening, Captain.
Most similar turn:
	 Good luck, Captain.
Response to most similar turn:
	 Don't need it, I'm a cat, I've got five lives.


In [0]:
# SBert predictions
sentence = " Good evening, Captain."
picked = nns_trf.nearest(sentence_vector_trnf(embedder, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

Your line:
	  Good evening, Captain.
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 Good evening.


In [0]:
# Get 5 nearest semantic responses
picked_list_nns_trf = nns_trf.nearest(sentence_vector_trnf(embedder, sentence), 5)
picked_list_nns = nns.nearest(sentence_mean(nlp, sentence), 5)

In [0]:
choice_count = 1
for choice in picked_list_nns:
  response_line_id = responses[choice]
  print(choice_count)
  print("Most similar turn:\n\t", movie_lines[choice])
  print("Response to most similar turn:\n\t", movie_lines[response_line_id])
  choice_count += 1

1
Most similar turn:
	 Good luck, Captain.
Response to most similar turn:
	 Don't need it, I'm a cat, I've got five lives.
2
Most similar turn:
	 Good luck, Captain.
Response to most similar turn:
	 I'm going too. They may need a doctor.
3
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 I come to the old lady's funeral.
4
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 Good evening, Mademoiselle.
5
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 Good evening.


In [0]:
choice_count = 1
for choice in picked_list_nns_trf:
  response_line_id = responses[choice]
  print(choice_count)
  print("Most similar turn:\n\t", movie_lines[choice])
  print("Response to most similar turn:\n\t", movie_lines[response_line_id])
  choice_count += 1



1
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 Good evening.
2
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 I come to the old lady's funeral.
3
Most similar turn:
	 Good evening, sir.
Response to most similar turn:
	 Good evening, Mademoiselle.
4
Most similar turn:
	 Good evening.
Response to most similar turn:
	 Hi.
5
Most similar turn:
	 Good evening.
Response to most similar turn:
	 Hi. You have hard shells, right?
